In [ ]:
from google.colab import drive
drive.mount('/content/drive2', force_remount=True)
driveHome = "/content/drive2/MyDrive/MasterThesis"


def flushDrive():
    drive.flush_and_unmount()
    drive.mount('/content/drive2')

Mounted at /content/drive2


In [ ]:
%%capture
!pip install youtube-dl moviepy
!pip install git+https://github.com/TahaAnwar/pafy.git#egg=pafy

!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [ ]:
import os
from os.path import basename, join
import cv2
import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
import re
import gc

from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)2146304/45929032 bytes (4.7%)4341760/45929032 bytes (9.5%)7462912/45929032 bytes (16.2%)10354688/45929032 bytes (22.5%)13426688/45929032 bytes (29.2%)16064512/45929032 bytes (35.0%)18751488/45929032 bytes (40.8%)21217280/45929032 bytes (46.2%)23805952/45929032 bytes (51.8%)26222592/45929032 bytes (57.1%)29253632/45929032 bytes (63.7%)32047104/45929032 bytes (69.8%)3

In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

IMAGE_HEIGHT , IMAGE_WIDTH = 224, 224
SEQUENCE_LENGTH = 20
DATASET_DIR = "UCF101"

CLASSES_LIST = [
           "BlowingCandles",
           "Billiards", 
           "BrushingTeeth", 
           "CuttingInKitchen",
           "Mixing",
           "MoppingFloor", 
           "ShavingBeard", 
           "Swing",
           "Typing",
           "WallPushups",
]

In [ ]:
if (not os.path.exists(DATASET_DIR)):
    Path(DATASET_DIR).mkdir(parents=True,exist_ok=True)
    root = os.path.join(driveHome,"UCF-101")
    for actionType in os.listdir(root):
        newActionPath=join(DATASET_DIR, actionType)
        Path(newActionPath).mkdir(exist_ok=True)
        moviesPath=join(root, actionType)
        lenList=len(os.listdir(moviesPath))
        print(f"{actionType} : {lenList}")
        counter = 0
        for movie in os.scandir(moviesPath):
            shutil.copyfile(movie.path, join(newActionPath, movie.name))
            counter += 1
            if counter >= lenList*1/2:
                break

BrushingTeeth : 131
CuttingInKitchen : 110
MoppingFloor : 110
ShavingBeard : 161
Typing : 136
WallPushups : 130
Billiards : 150
Mixing : 136
Swing : 131
BlowingCandles : 109


In [ ]:
CHOSEN_MODELS={
    "InceptionV3": [
                    "/content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.1/greyscale/InceptionV3Model/250/model_ex-011_loss-0.740166_acc-0.732337.h5",
                    "/content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.1/mix-net/InceptionV3Model/2000/model_ex-008_loss-0.295730_acc-0.901495.h5",
                    ],
    "MobileNetV2" : [
                    "/content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.1/brightness/MobileNetV2Model/250/model_ex-029_loss-1.070119_acc-0.591033.h5",
                     "/content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.333/mix-weak/MobileNetV2Model/3000/model_ex-020_loss-0.561358_acc-0.828635.h5",
                    ],
    "EfficientNetV2B0" : [
                        "/content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.1/gaussian-mix/EfficientNetV2B0Model/250/model_ex-011_loss-0.749594_acc-0.729959.h5",
                        "/content/drive2/MyDrive/MasterThesis/taught-smart/-with-transfer/Augmented/0.1/crop/EfficientNetV2B0Model/2000/model_ex-003_loss-0.179559_acc-0.958050.h5",
                        ], 
    }

In [ ]:
def frames_extraction(video_path, normalize):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read() 
        if not success:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame
        if normalize:
            normalized_frame = resized_frame / 255
        frames_list.append(normalized_frame)
    video_reader.release()

    return frames_list

In [ ]:
def create_dataset(normalize):
    features = []
    labels = []
    video_files_paths = []
    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            frames = frames_extraction(video_file_path, normalize)
            if len(frames) == SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    features = np.asarray(features)
    labels = np.array(labels)  
    
    return features, labels, video_files_paths

In [ ]:
def prepare_data(normalize):
    features, labels, video_files_paths = create_dataset(normalize)
    one_hot_encoded_labels = to_categorical(labels)
    features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels,
                                                                            test_size = 0.25, shuffle = True,
                                                                            random_state = seed_constant)
    
    return features_train, features_test, labels_train, labels_test

In [ ]:
def create_LRCN_model(path_to_model, rnn):
    base_model=keras.models.load_model(path_to_model)
    model = Sequential()
    model.add(TimeDistributed(base_model, input_shape=(SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    model.layers[0].trainable=False
    if rnn == "GRU":
        model.add(tf.keras.layers.GRU(64))
    else:
        model.add(tf.keras.layers.LSTM(64))
    model.add(tf.keras.layers.Dense(len(CLASSES_LIST), activation='softmax', use_bias=True))

    return model

In [ ]:
def save(LRCN_model, modelName, number, features_test, labels_test, rnn_type):
    model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)
    model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history
    date_time_format = '%Y_%m_%d__%H_%M_%S'
    current_date_time_dt = dt.datetime.now()
    current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
    model_file_name = f'{driveHome}/UCF-SmartSplit/moviesModels/LRCN_model__{modelName}_{number}_{rnn_type}_Date_Time_{current_date_time_string}___Accuracy_{model_evaluation_accuracy}.h5'

    LRCN_model.save(model_file_name)

In [ ]:
def train(LRCN_model, features_train, labels_train):
    early_stopping_callback = EarlyStopping(monitor = 'val_accuracy', patience = 10, restore_best_weights = True)
    LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
    LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 100, batch_size = 4,
                                                shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])
    return LRCN_model

In [ ]:
for modelName in CHOSEN_MODELS.keys():
    if modelName == "EfficientNetV2B0":
        features_train, features_test, labels_train, labels_test = prepare_data(False)
    else:
        features_train, features_test, labels_train, labels_test = prepare_data(True)

    for rnn_type in ["GRU", "LSTM"]:
        for idx, modelFullPath in enumerate(CHOSEN_MODELS[modelName]):
            kerasModel = create_LRCN_model(modelFullPath, rnn_type)
            print(modelName, rnn_type, idx)
            kerasModel = train(kerasModel, features_train, labels_train)
            save(kerasModel, modelName, idx, features_test, labels_test, rnn_type)
            flushDrive()
    
    del features_train
    gc.collect()

Extracting Data of Class: BlowingCandles
Extracting Data of Class: Billiards
Extracting Data of Class: BrushingTeeth
Extracting Data of Class: CuttingInKitchen
Extracting Data of Class: Mixing
Extracting Data of Class: MoppingFloor
Extracting Data of Class: ShavingBeard
Extracting Data of Class: Swing
Extracting Data of Class: Typing
Extracting Data of Class: WallPushups
InceptionV3 0
Epoch 1/100
98/98 [==============================] - 25s 162ms/step - loss: 1.8729 - accuracy: 0.5612 - val_loss: 1.4234 - val_accuracy: 0.6327
Epoch 2/100
98/98 [==============================] - 12s 125ms/step - loss: 0.9529 - accuracy: 0.7143 - val_loss: 0.8054 - val_accuracy: 0.7041
Epoch 3/100
98/98 [==============================] - 12s 125ms/step - loss: 0.7074 - accuracy: 0.7270 - val_loss: 0.6515 - val_accuracy: 0.7143
Epoch 4/100
98/98 [==============================] - 12s 125ms/step - loss: 0.5955 - accuracy: 0.7577 - val_loss: 0.5767 - val_accuracy: 0.8061
Epoch 5/100
98/98 [=================

In [ ]:
before_merge = join(driveHome, "movies_to_merge")
fileNamesInSpecialOrder = ["WallPushups.avi", "BlowingCandles.avi", "CuttingInKitchen.avi", "Mixing.avi", "Swing.avi"]
video_frames_quantities = []
for fileName in fileNamesInSpecialOrder:
    video_reader = cv2.VideoCapture(join(before_merge, fileName))
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    video_frames_quantities.append(video_frames_count)

    video_reader.release()

video_classes_treshold = [0 for k in range(len(fileNamesInSpecialOrder))]

for idx, frames_count in enumerate(video_frames_quantities):
    for i in range(idx, len(fileNamesInSpecialOrder)):
        video_classes_treshold[i] += frames_count

print(video_frames_quantities)
print(video_classes_treshold)

[68, 130, 194, 124, 151]
[68, 198, 392, 516, 667]


In [ ]:
def frames_counter_to_ground_truth(frameNumber):
    label = ""
    fake = False
    for idx, treshold in enumerate(video_classes_treshold):
        if frameNumber < treshold:
            label = fileNamesInSpecialOrder[idx][:-4]

            if idx > 0:
                if frameNumber < video_classes_treshold[idx-1] + 20:
                    fake = True
            else:
                if frameNumber < 20:
                    fake = True
            
            break
    
    
    return label, fake

In [ ]:
#[68, 198, 392, 516, 667]

In [ ]:
def predict_on_video(video_file_path, output_file_path, SEQUENCE_LENGTH, kerasModel, kerasModelName):
    resolving_frame_number = 0
    counted_frames = 0
    good_frames = 0
    is_fake = False
    video_reader = cv2.VideoCapture(video_file_path)
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 
                                   video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))
    frames_queue = deque(maxlen = SEQUENCE_LENGTH)
    predicted_class_name = ""
    while video_reader.isOpened():
        ok, frame = video_reader.read() 
        if not ok:
            break
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))

        normalized_frame = resized_frame
        if not kerasModelName == "EfficientNetV2B0":
            normalized_frame = resized_frame / 255
        frames_queue.append(normalized_frame)
        if len(frames_queue) == SEQUENCE_LENGTH:
            predicted_labels_probabilities = kerasModel.predict(np.expand_dims(frames_queue, axis = 0))[0]
            predicted_label = np.argmax(predicted_labels_probabilities)
            predicted_class_name = CLASSES_LIST[predicted_label]
         
        resolving_frame_number += 1
        ground_truth, is_fake = frames_counter_to_ground_truth(resolving_frame_number)
        cv2.putText(frame, ground_truth, (10, 150), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 255, 0), 2)
        cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 2)
        video_writer.write(frame)

        if not is_fake:
            counted_frames += 1
            if ground_truth == predicted_class_name:
                good_frames +=1

    video_reader.release()
    video_writer.release()

    return good_frames/counted_frames

In [ ]:
moviePath = join(driveHome,"movie_to_test.mp4")
movieNewPath = os.path.basename(moviePath)
shutil.copyfile(moviePath, join(movieNewPath))

'movie_to_test.mp4'

In [ ]:
trainedModelsPath = f"{driveHome}/UCF-SmartSplit/moviesModels/"
path_forNewMovies = f"{driveHome}/UCF-SmartSplit/newMovies/"

In [ ]:
Path(path_forNewMovies).mkdir(parents=True, exist_ok=True)
with open(join(driveHome, "UCF-SmartSplit", "GoodFrames.csv"), "w") as f: 
    f.write("model_name,ratio\n")
    for path in os.listdir(trainedModelsPath):
        modelPath = join(trainedModelsPath, path)
        kerasModel=keras.models.load_model(modelPath)
        modelName = re.search("EfficientNetV2B0|InceptionV3|MobileNetV2", path).group()
        print(modelName)
        output_video_file_path = f"{path_forNewMovies}{path}-Output.mp4"
        good_to_bad_ratio = predict_on_video(movieNewPath, output_video_file_path, SEQUENCE_LENGTH, kerasModel, modelName)
        f.write(f"{path},{good_to_bad_ratio}\n")

flushDrive()

InceptionV3


In [ ]:
def get_images_examples(video_file_path, output_file_path):
    Path(output_file_path).mkdir(parents=True, exist_ok=True)
    video_reader = cv2.VideoCapture(video_file_path)
    wanted_frames = [65, 199, 490, 640]
    for frame_count in wanted_frames:
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_count)
        ok, frame = video_reader.read() 
        if not ok:
            break

        wrote = cv2.imwrite(join(output_file_path, str(frame_count) + ".jpg"), frame)
        if not wrote:
            print("Sth went wrong")

    video_reader.release()

In [ ]:
video_file_path=os.listdir(path_forNewMovies)[1]
video_file_path = join(path_forNewMovies, video_file_path)
output_file_path=f"{driveHome}/UCF-SmartSplit/classifiedFrames/"
get_images_examples(video_file_path, output_file_path)

In [ ]:
flushDrive()